In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import os

import pandas as pd

from torch.utils.data import Dataset

from dataloader import AllBeautyLoader

# AllBeauty для BERT

<b>Особый класс датасета</b> нужен для его удобного дальнейшего использования при подготовке моделей на базе BERT

In [2]:
# AllBeauty for BERT
class AllBeautyBERTDataset(Dataset):
    def __init__(self, path: str, file_format: str):
        """
        path: Путь к файлу с данными. Это строка вида '/home/user/docs/filename.csv'
        file_format: Формат входного файла. Должен быть либо 'csv', либо 'xlsx' 
        """

        self._allowed_formats = ('csv', 'xlsx')
        self._form_df = {
            'xlsx': pd.read_excel,
            'csv': pd.read_csv
        }
        
        if not (isinstance(path, str) and os.path.exists(path)):
            raise ValueError('Path must be a string and path must exists!')

        if not (isinstance(file_format, str) and file_format in self._allowed_formats):
            raise ValueError(f'File format must be a string and must be one of the next type: {self._allowed_formats}')

        self.df = self._form_df[file_format](path)
        self.size = self.df.shape
        
    def __len__(self):
        return self.size[0]

    def __getitem__(self, index: int):
        if not (isinstance(index, int) and 0 <= index < self.size[0]):
            raise ValueError(f'Index must be ab integer and must be lay in [0; {self.size[0]})')

        return self.df.loc[index]

# Пример создания датасета AllBeauty для BERT-а

In [3]:
# настроим пути входящих и выходящих файлов
input_path = '/home/roman/Документы/AmazonRecomendationSystem/data/raw/all_beauty/All_Beauty.jsonl'
output_path = os.getcwd()

In [4]:
# Создаем экземпляр класса, укажем свой размер выборки и свое семя случайной генерации
loader = AllBeautyLoader(input_path, sample_size=1000, seed=12)

In [5]:
# Получаем выборку данных
sample_df = loader.get_sample()

# Просматриваем первые пять строк выборки
sample_df.head()

Загрузка данных: 100%|███████████████████| 1000/1000 [00:00<00:00, 24770.29it/s]


,rating,title,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
472907,5.0,Worth the money- don’t go cheap- for serious u...,Love this brush after so many that were not wo...,B075B3M8F4,B075B3M8F4,AFTRHI7BZD35G7ZFDF5HWXV5OPEQ,1582230773060,1,True
255366,5.0,Nice eye cream.,I'm not seeing my wrinkles evaporate but it so...,B00N996X4G,B00N996X4G,AETK7DFLGHKINK5CS5ISOCTHVZPQ,1460144590000,0,True
412925,5.0,Keep supplying my hair product.,"I dont know if sauve discontinued this line, b...",B0185LCXKA,B0185LCXKA,AGQU3SLLI72FLEDMOON2EP7WAN5Q,1543890519281,0,True
564465,5.0,Five Stars,very pretty bows and good price for them,B00RVNUPXQ,B00RVNUPXQ,AGRPAEACQYPMCQEUGFLYQIKTALTQ,1444833618000,0,True
36482,2.0,Only good for cornrow install!,I love the way this hair looks. The color and ...,B07H4L1XBY,B07H4L1XBY,AH7GI3HY7NK4INJEPUZBHX43LUHQ,1561653660569,0,True


In [6]:
loader.save_sample(output_path, 'csv')

Данные уже существуют


In [7]:
dataset = AllBeautyBERTDataset(os.path.join(os.getcwd(), 'sample_size_1000_seed_12_used_True.csv'), 'csv')

In [8]:
len(dataset)

1000

In [9]:
dataset[12]

Unnamed: 0                                                      696738
rating                                                             3.0
title                                                      Three Stars
text                 Bit are ok quality but they are NOT CARBIDE TH...
asin                                                        B01LE4QJS2
parent_asin                                                 B01LE4QJS2
user_id                                   AETZHC6HNADDF75I2TSFES5B2K3Q
timestamp                                                1497738860179
helpful_vote                                                        16
verified_purchase                                                 True
Name: 12, dtype: object

In [10]:
type(dataset[12])

pandas.core.series.Series